In [25]:
#tải các thư viện cần thiết
!pip install transformer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer


In [26]:
import numpy as np
import pandas as pd
import torch

#Embedding
from transformers import BertTokenizerFast,BertModel

#Modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

In [27]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased') 
model = BertModel.from_pretrained("bert-base-uncased")

In [28]:
def read_dataset(file_path):
    tokens=[]
    ner_tags=[]
    ids=[]
    count=1
    with open(file_path) as f:
        lines=f.readlines()
        ts=[]
        nts=[]
        for line in lines:
            line = line.split()
            if len(line)==0:
                ids.append(count)
                tokens.append(ts)
                ner_tags.append(nts)
                ts=[]
                nts=[]
                count+=1
            else:
                ts.append(line[0])
                nts.append(line[-1])
    data = pd.DataFrame({'Id':ids, 'NER_tags':ner_tags, 'Tokens':tokens})
    return data

In [29]:
def get_NER_labels(data):
    NERs = list(data['NER_tags'].values)
    labels_list = []
    for value in NERs:
        labels_list = labels_list + value
    types = list(set(labels_list))
    y = [types.index(value) for value in labels_list]
    return np.array(y), types

In [30]:
def align_shape(emb_matrix, word_ids):
    result = []
    unique_idx = set(word_ids)
    processed_rows = set()

    for i, row in enumerate(emb_matrix):
        if i not in processed_rows:
            current_value = word_ids[i]
            matching_indices = [i]
            avg_row = np.array(row, dtype=float)
            for j in range(i + 1, len(emb_matrix)):
                if word_ids[j] == current_value:
                    matching_indices.append(j)
                    avg_row += np.array(emb_matrix[j], dtype=float)
                    processed_rows.add(j)
            avg_row /= len(matching_indices)
            result.append(avg_row.tolist())

    return result

In [31]:
def downsize_block(tensor):
    input_dim = tensor.shape[1]
    linear1 = torch.nn.Linear(input_dim, 512)
    linear2 = torch.nn.Linear(512, 256)
    linear3 = torch.nn.Linear(256, 128)

    tensor = linear1(tensor)
    tensor = linear2(tensor)
    tensor = linear3(tensor)

    return tensor

In [32]:
def get_features(data, tokenizer, model):
    samples=[]
    for i in range(len(data)):
        tokenized_input = tokenizer(data.iloc[i][2], add_special_tokens=True,
                                    padding=False, return_tensors="pt", is_split_into_words=True)
        output=model(**tokenized_input)
        last_hidden_state = output[0][:,1:-1,:] #remove embedding of special tokens
        last_hidden_state = last_hidden_state.detach().squeeze(0).numpy().tolist()
        last_hidden_state = align_shape(last_hidden_state, tokenized_input.word_ids()[1:-1])
        samples = samples + last_hidden_state
    return np.array(samples)

# **MÔ HÌNH HÓA CHO TẬP CONLL**

In [33]:
train_data = read_dataset('/kaggle/input/conll-2003/train.txt')

In [34]:
train_data.head()

,Id,NER_tags,Tokens
0,1,"[B-ORG, O, B-MISC, O, O, O, B-MISC, O, O]","[EU, rejects, German, call, to, boycott, Briti..."
1,2,"[B-PER, I-PER]","[Peter, Blackburn]"
2,3,"[B-LOC, O]","[BRUSSELS, 1996-08-22]"
3,4,"[O, B-ORG, I-ORG, O, O, O, O, O, O, B-MISC, O,...","[The, European, Commission, said, on, Thursday..."
4,5,"[B-LOC, O, O, O, O, B-ORG, I-ORG, O, O, O, B-P...","[Germany, 's, representative, to, the, Europea..."


In [35]:
X_train = get_features(train_data, tokenizer, model)
y_train, ners_list = get_NER_labels(train_data)

In [36]:
X_train.shape

(203617, 768)

In [37]:
y_train.shape

(203617,)

In [38]:
model_for_ner = RandomForestClassifier(min_samples_split=2,bootstrap=True,oob_score=True,
                                      max_samples=int(0.9*len(train_data)))

In [39]:
param_grid = {
    'n_estimators': [10, 20, 30],
    'criterion': ['gini', 'entropy']
}

best_model_for_ner = GridSearchCV(model_for_ner, param_grid, cv=5, scoring='f1_macro')
best_model_for_ner.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(max_samples=12636,
                                              oob_score=True),
             param_grid={'criterion': ['gini', 'entropy'],
                         'n_estimators': [10, 20, 30]},
             scoring='f1_macro')

In [40]:
# In ra tham số tốt nhất và điểm số tốt nhất
print("Tham số tốt nhất:", best_model_for_ner.best_params_)
print("Điểm số tốt nhất:", best_model_for_ner.best_score_)

Tham số tốt nhất: {'criterion': 'entropy', 'n_estimators': 20}
Điểm số tốt nhất: 0.36511305538790584


In [41]:
#accuracy trên tập validation cho mỗi phần dự đoán
print('Acc cho những mẫu không được huấn luyện theo OOB score của RandomForest là: ',
      best_model_for_ner.best_estimator_.oob_score_)

Acc cho những mẫu không được huấn luyện theo OOB score của RandomForest là:  0.8726137797924535


In [42]:
#Đánh giá accuracy trên tập test
test_data = read_dataset('/kaggle/input/conll-2003/test.txt')
X_test = get_features(test_data, tokenizer, model)
y_test,_ = get_NER_labels(test_data)

In [43]:
predict_ners = best_model_for_ner.predict(X_test)

In [44]:
predict_labels = [ners_list[i] for i in predict_ners]

In [45]:
print(classification_report(y_pred=predict_ners, y_true=y_test,target_names=ners_list))

              precision    recall  f1-score   support

       I-LOC       1.00      0.08      0.15       257
       I-ORG       0.75      0.03      0.05       834
       B-ORG       0.03      0.01      0.01      1616
       I-PER       0.84      0.26      0.40      1156
       B-PER       0.02      0.01      0.01      1660
      B-MISC       0.65      0.07      0.13       701
      I-MISC       0.20      0.01      0.02       214
           O       0.86      1.00      0.93     38289
       B-LOC       0.85      0.35      0.49      1667

    accuracy                           0.85     46394
   macro avg       0.58      0.20      0.24     46394
weighted avg       0.79      0.85      0.80     46394



- Các Precision ở các lớp tuy cao nhưng Recall lại thấp chứng tỏ mô hình RandomForest đang bỏ sót khá nhiều các mẫu thuộc một lớp nhất định và việc dự đoán nhầm không tập trung vào một lớp cụ thể nào hết mà phân bố đều trên 8 lớp.

# **MÔ HÌNH HÓA CHO TẬP COVID19**

## Mức Syllable

In [46]:
train_data = read_dataset('/kaggle/input/covid19vi/syllable/train_syllable.conll')

In [47]:
train_data.head()

,Id,NER_tags,Tokens
0,1,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Đồng, thời, ,, bệnh, viện, tiếp, tục, thực, h..."
1,2,"[O, O, O, O, O, O, O, O, O, O, O, B-SYMPTOM_AN...","["", Số, bệnh, viện, có, thể, tiếp, nhận, bệnh,..."
2,3,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Ngoài, ra, ,, những, người, tiếp, xúc, gián, ..."
3,4,"[O, O, O, O, O, O, O, B-LOCATION, O, B-LOCATIO...","[Bà, này, khi, trở, về, quá, cảnh, Doha, (, Qa..."
4,5,"[O, O, O, B-PATIENT_ID, O, O, O, O, O, O, O, B...","["", Bệnh, nhân, 523, "", và, chồng, là, "", bệnh..."


In [48]:
X_train = get_features(train_data, tokenizer, model)
y_train, ners_list = get_NER_labels(train_data)

In [49]:
model_for_ner_vi_syll = RandomForestClassifier(n_estimators=20,criterion='entropy',min_samples_split=2,bootstrap=True,oob_score=True,
                                      max_samples=int(0.9*len(train_data)))

In [50]:
model_for_ner_vi_syll.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_samples=4524, n_estimators=20,
                       oob_score=True)

In [51]:
#Đánh giá trên tập test
test_data = read_dataset('/kaggle/input/covid19vi/syllable/test_syllable.conll')
X_test = get_features(test_data, tokenizer, model)
y_test,_ = get_NER_labels(test_data)

In [52]:
predict_ners = model_for_ner_vi_syll.predict(X_test)

In [53]:
print(classification_report(y_pred=predict_ners, y_true=y_test,target_names=ners_list))

                       precision    recall  f1-score   support

                I-JOB       0.00      0.00      0.00       347
           B-LOCATION       0.83      0.06      0.12      4441
       B-ORGANIZATION       1.00      0.01      0.01       771
               I-DATE       0.96      0.87      0.92      1752
             I-GENDER       0.00      0.00      0.00         1
         B-PATIENT_ID       0.77      0.77      0.77      2005
     I-TRANSPORTATION       0.00      0.00      0.00        72
     B-TRANSPORTATION       1.00      0.14      0.25       193
                    O       0.76      0.99      0.86     77772
                I-AGE       0.00      0.00      0.00         6
           I-LOCATION       0.82      0.10      0.18     10729
         I-PATIENT_ID       0.00      0.00      0.00        27
                B-AGE       0.96      0.15      0.26       582
               B-DATE       0.96      0.74      0.84      1654
             B-GENDER       1.00      0.24      0.39  

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Mức 2-Words

In [54]:
train_data = read_dataset('/kaggle/input/covid19vi/word/train_word.conll')

In [55]:
X_train = get_features(train_data, tokenizer, model)
y_train, ners_list = get_NER_labels(train_data)

In [56]:
model_for_ner_vi_word = RandomForestClassifier(n_estimators=20,criterion='entropy',min_samples_split=2,bootstrap=True,oob_score=True,
                                      max_samples=int(0.9*len(train_data)))

In [57]:
model_for_ner_vi_word.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_samples=4524, n_estimators=20,
                       oob_score=True)

In [58]:
test_data = read_dataset('/kaggle/input/covid19vi/word/test_word.conll')
X_test = get_features(test_data, tokenizer, model)
y_test,_ = get_NER_labels(test_data)

In [59]:
predict_ners = model_for_ner_vi_word.predict(X_test)

In [60]:
print(classification_report(y_pred=predict_ners, y_true=y_test,target_names=ners_list))

                       precision    recall  f1-score   support

                I-JOB       0.00      0.00      0.00       114
           B-LOCATION       0.70      0.12      0.20      4441
       B-ORGANIZATION       0.95      0.05      0.10       771
               I-DATE       0.96      0.91      0.94      1750
         B-PATIENT_ID       0.79      0.77      0.78      2005
     I-TRANSPORTATION       0.00      0.00      0.00        69
     B-TRANSPORTATION       1.00      0.07      0.13       193
                    O       0.80      1.00      0.89     63545
                I-AGE       0.00      0.00      0.00         6
           I-LOCATION       0.82      0.07      0.12      4918
         I-PATIENT_ID       0.00      0.00      0.00        27
                B-AGE       0.94      0.20      0.33       582
               B-DATE       0.94      0.81      0.87      1654
             B-GENDER       1.00      0.38      0.55       462
               B-NAME       1.00      0.05      0.09  

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
